# Imports

In [1]:
import numpy as np
import itertools
import time
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
# import pixiedust
from numba import njit, float32, void
import os
import cProfile as profile

# Introduction

Consider the following equation:
$$\Delta_p \Psi(x,y)=f(x,y)$$

$x \in [0,1],\ y \in [0,1]$ with *Dirichlet* BC: $\Psi(0,y) = 0$, $\Psi(1,y) = 0$, $\Psi(x,0) = 0$ and $\Psi(x,1) = 0$.

For this first attempt, we will take $p=2$.

# Defining functions

## Sigmoid

Sigmoid $\sigma(x) = \frac{1}{1+e^{-x}}$ and its derrivatives.

Sigmoid with parameter $t$
$$ \sigma_t(x) = \frac{1}{1+e^{-tx}}$$

In [2]:
T = 1

In [3]:
@njit
def sig(x):
    return 1 / (1 + np.exp(-T*x))

@njit
def sig1(x):
    return T * sig(x) * (1 - sig(x))

@njit
def sig2(x):
    return T * (sig1(x) - 2*sig(x)*sig1(x))

@njit
def sig3(x):
    return T * (sig2(x) - 2 * (sig1(x)**2 + sig(x) * sig2(x)))

@njit
def sig_pr(x, k):
    if k==0:
        return sig(x)
    if k==1:
        return sig1(x)
    if k==2:
        return sig2(x)
    if k==3:
        return sig3(x)

## RHS

The right side of the equation:  $\Delta\Psi(x,y) = 6 x (x-1) (1-2 y) + 2 y (y-1) (1-2 y)$

In [4]:
K=10
@njit
def f(x, y):
    return K * (6*(x-1)*x*(1-2*y) + 2*(y-1)*y*(1-2*y))

The analytic solution is given by: $\ \Psi_a(x,y) = x (1-x) y (1-y) (1-2 y)$

In [5]:
@njit
def psi_a(x, y):
    return K * x*(1-x)*y*(1-y)*(1-2*y)

$$\frac{\partial}{\partial x} \Psi_a = (1-2x) y (1-y) (1-2 y)$$

In [6]:
@njit
def psi_a_dx(x,y):
    return K * (1-2*x)*y*(1-y)*(1-2*y)

$$\frac{\partial}{\partial y} \Psi_a = x (1-x) (1-6y + 6y^2) $$

In [7]:
@njit
def psi_a_dy(x,y):
    return K * x*(1-x)*(1-6*y+6*(y**2))

## Neural Network

The output of neural network $N(x,y,\vec{p})$, where $\vec{p} = [w, u, v]$:
$$N = \sum_i^H v_i \sigma(z_i) \text{, where } z_i = w_{i0} x + w_{i1} y + u_i$$

In [8]:
@njit
def z(x, y, p00, p01, p1):
# def z(x, y, p00, p01, p1, z_res):
    z_x = x * p00
    z_y = y * p01
    z_ = z_x + z_y + p1
#     for i in range(p00.shape[0]):
#         z_res[i] = x*p00[i] + y*p01[i] + p1[i]

    return z_

@njit
def N(x, y, p00, p01, p1, p2):
    return np.sum(sig(z(x, y, p00, p01, p1)) * p2)

$$\frac{\partial^k N}{\partial x_j^k} = \sum_{i=1}^H v_i w_{ij}^k \sigma^{(k)}$$

In [9]:
@njit
def dN_dxj_k(x, y, p00, p01, p1, p2, j, k):
    wj = p00 if j==0 else p01
    z_ = z(x, y, p00, p01, p1)
    
    return np.sum(p2 * (wj**k) * sig_pr(z_, k))

$$\frac{\partial N}{\partial w_j} = x_j v \sigma '$$

In [10]:
@njit
def dN_dwj(x, y, p00, p01, p1, p2, j):
    xj = x if j==0 else y
    z_ = z(x, y, p00, p01, p1)
    return xj * p2 * sig1(z_)

$$ \frac{\partial}{\partial w_j} \frac{\partial N}{\partial x_k} = x_j v w_k \sigma'' + v_i \sigma' \quad\text{ if } j = k$$

$$ \frac{\partial}{\partial w_j} \frac{\partial N}{\partial x_k} = x_j v w_k \sigma'' \quad\text{ if } j \neq k$$

In [11]:
@njit
def d_dwj_dN_dxk(x, y, p00, p01, p1, p2, j, k):
    xj = x if j==0 else y
    wk = p00 if k==0 else p01
    jk = 1 if j==k else 0
    z_ = z(x, y, p00, p01, p1)
    return xj * p2 * wk * sig2(z_) + jk * p2 * sig1(z_)

$$ \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x_k^2} = x_j v w_k^2 \sigma^{(3)} + 2 v w_k \sigma'' \quad\text{ if } j = k $$

$$ \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x_k^2} = x_j v w_k^2 \sigma^{(3)} \quad\text{ if } j \neq k $$

In [12]:
@njit
def d_dwj_dN2_dxk2(x, y, p00, p01, p1, p2, j, k):
    xj = x if j==0 else y
    wk = p00 if k==0 else p01
    jk = 1 if j==k else 0
    z_ = z(x, y, p00, p01, p1)
    return xj * p2 * (wk**2) * sig3(z_) + jk * 2 * p2 * wk * sig2(z_)

$$ \frac{\partial}{\partial u} \frac{\partial^k}{\partial x_j^k} N = v w_j^k \sigma^{(k+1)} $$

In [13]:
@njit
def d_du_dkN(x, y, p00, p01, p1, p2, j, k):
    wj = p00 if j==0 else p01
    z_ = z(x, y, p00, p01, p1)
    return p2 * (wj**k) * sig_pr(z_, k+1)

$$ \frac{\partial}{\partial v} \frac{\partial^k}{\partial x_j^k} N = w_j^k \sigma^{(k)} $$

In [14]:
@njit
def d_dv_dkN(x, y, p00, p01, p1, p2, j, k):
    wj = p00 if j==0 else p01
    z_ = z(x, y, p00, p01, p1)
    return (wj**k) * sig_pr(z_, k)

## Cost function

$$E[\vec{p}] = \sum_{i \in \hat{D}} \left\{ \frac{\partial^2 N}{\partial x^2} + \frac{\partial^2 N}{\partial y^2} - f(x,y) \right\}^2 
           +  \sum_{i \in \partial \hat{D}} N^2$$

In [15]:
BC=1

In [16]:
@njit# ('float64(float64, float64, float64[:], float64[:], float64[:], float64[:])')
def error_term1(x, y, p00, p01, p1, p2):
    ans = dN_dxj_k(x, y, p00, p01, p1, p2, 0, 2) + \
            dN_dxj_k(x, y, p00, p01, p1, p2, 1, 2)  -  f(x, y)
    return ans

In [17]:
@njit
def cost(points, boundary_points, p00, p01, p1, p2):
    et1 = np.zeros(points.shape[0])
    et2 = np.zeros(boundary_points.shape[0])
    
    for i, pnt in enumerate(points):
        et1[i] = error_term1(pnt[0], pnt[1], p00, p01, p1, p2)**2
    
    for i, pnt in enumerate(boundary_points):
        et2[i] = N(pnt[0], pnt[1], p00, p01, p1, p2)**2
    
    cost = np.sum(et1) + BC*np.sum(et2)
    
    return cost

In [18]:
def relative_err_without_points(p00, p01, p1, p2, nx=100):
    all_points = np.array(list(itertools.product(np.linspace(0, 1, nx), np.linspace(0, 1, nx))))
    return relative_err(p00, p01, p1, p2, all_points)

@njit
def relative_err(p00, p01, p1, p2, all_points):
    dOmega = 1. / len(all_points)
    
    tr1 = np.zeros(all_points.shape[0])
    tr2 = np.zeros(all_points.shape[0])
    ana1 = np.zeros(all_points.shape[0])
    ana2 = np.zeros(all_points.shape[0])
    
    for i, pnt in enumerate(all_points):
        tr1[i] = dOmega * (np.abs(N(pnt[0], pnt[1], p00, p01, p1, p2) - psi_a(pnt[0],pnt[1]))**2)

    for i, pnt in enumerate(all_points):
        tr2[i] = dOmega * ((dN_dxj_k(pnt[0],pnt[1],p00,p01,p1,p2,0,1)-psi_a_dx(pnt[0],pnt[1]))**2 +
                           (dN_dxj_k(pnt[0],pnt[1],p00,p01,p1,p2,1,1)-psi_a_dy(pnt[0],pnt[1]))**2)

    for i, pnt in enumerate(all_points):
        ana1[i] = dOmega * (np.abs(psi_a(pnt[0],pnt[1]))**2)
        
    for i, pnt in enumerate(all_points):
        ana2[i] = dOmega* (psi_a_dx(pnt[0],pnt[1])**2 + psi_a_dy(pnt[0],pnt[1])**2)
    
    rel_err = (np.sum(tr1) + np.sum(tr2))**(1/2) / (np.sum(ana1) + np.sum(ana2))**(1/2)

    return rel_err

# Gradients

$$ \frac{\partial E[\vec{p}]}{\partial w_j} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial w_j} \frac{\partial^2 N}{\partial y^2} \right) \right\}  +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial w_j}$$

In [19]:
@njit
def dE_dwj(points, boundary_points, p00, p01, p1, p2, j):
    t1 = np.zeros((points.shape[0], p00.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p00.shape[0]))
    
    for i, pnt in enumerate(points):
        t1[i] = 2 * error_term1(pnt[0],pnt[1],p00, p01, p1, p2) * (
                d_dwj_dN2_dxk2(pnt[0],pnt[1],p00, p01, p1, p2,j,0) + 
                d_dwj_dN2_dxk2(pnt[0],pnt[1],p00, p01, p1, p2,j,1))
        
    for i, pnt in enumerate(boundary_points):
        t2[i] = 2 * N(pnt[0],pnt[1],p00, p01, p1, p2) * dN_dwj(pnt[0],pnt[1],p00, p01, p1, p2,j)
    
    grad = np.sum(t1, axis=0) + BC*np.sum(t2, axis=0)
    
    return grad

$$ \frac{\partial E[\vec{p}]}{\partial u} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial u} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial u} \frac{\partial^2 N}{\partial y^2} \right) \right\} +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial u}$$

In [20]:
@njit
def dE_du(points, boundary_points, p00, p01, p1, p2):
    t1 = np.zeros((points.shape[0], p1.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p1.shape[0]))
    
    for i, pnt in enumerate(points):
        t1[i] = 2 * error_term1(pnt[0],pnt[1],p00, p01, p1, p2) * \
                (d_du_dkN(pnt[0],pnt[1],p00, p01, p1, p2,0,2) + \
                 d_du_dkN(pnt[0],pnt[1],p00, p01, p1, p2,1,2))
        
    for i, pnt in enumerate(boundary_points):
        t2[i] = 2 * N(pnt[0],pnt[1],p00, p01, p1, p2) * d_du_dkN(pnt[0],pnt[1],p00, p01, p1, p2,0,0)
    
    grad = np.sum(t1, axis=0) + BC*np.sum(t2, axis=0)
    
    return grad

$$ \frac{\partial E[\vec{p}]}{\partial v} = \sum_{i \in \hat{D}} \left\{ 2 \text{ (error_term1) } \left( \frac{\partial}{\partial v} \frac{\partial^2 N}{\partial x^2} + \frac{\partial}{\partial v} \frac{\partial^2 N}{\partial y^2} \right) \right\}  +  \sum_{i \in \partial \hat{D}} 2 N \frac{\partial N}{\partial v}$$

In [21]:
@njit
def dE_dv(points, boundary_points, p00, p01, p1, p2):
    t1 = np.zeros((points.shape[0], p2.shape[0]))
    t2 = np.zeros((boundary_points.shape[0], p2.shape[0]))
    
    for i, pnt in enumerate(points):
        t1[i] = 2 * error_term1(pnt[0],pnt[1],p00, p01, p1, p2) * \
                (d_dv_dkN(pnt[0],pnt[1],p00, p01, p1, p2,0,2) + \
                 d_dv_dkN(pnt[0],pnt[1],p00, p01, p1, p2,1,2))
        
    for i, pnt in enumerate(boundary_points):
        t2[i] = 2 * N(pnt[0],pnt[1],p00, p01, p1, p2) * d_dv_dkN(pnt[0],pnt[1],p00, p01, p1, p2,0,0)
    
    grad = np.sum(t1, axis=0) + BC*np.sum(t2, axis=0)
    
    return grad

# NN class

In [22]:
# @njit
def get_mini_batches(points, boundary_points, batch_size):
    np.random.shuffle(points)
    np.random.shuffle(boundary_points)
    
    no_of_splits = np.ceil( (len(points) + len(boundary_points)) / batch_size)

    mini_batch_points = np.array_split(points, no_of_splits)
    mini_batch_boundary_points = np.array_split(boundary_points, no_of_splits)
    
    return mini_batch_points, mini_batch_boundary_points

In [23]:
class NNTrain:
    def __init__(self, nx=10, bx=10, hidden_nodes=10, alpha=0.01, batch_size=50,
                 beta=0.9, if_rel_err=False):
        
        self.nx = nx
        self.hidden_nodes = hidden_nodes
        self.alpha = alpha
        self.batch_size = batch_size
        self.beta = beta
        self.boundary_points = np.array(list(set(list(itertools.product([0, 1], np.linspace(0,1,bx))) +
                                                 list(itertools.product(np.linspace(0,1,bx), [0, 1])))))
        pnts = list(itertools.product(np.linspace(0, 1, nx), np.linspace(0, 1, nx)))
        self.points = np.array([(x,y) for x,y in pnts if (x not in [0,1] and y not in [0,1])])
        self.cost_rate = []
        self.if_rel_err = if_rel_err
        if self.if_rel_err:
            self.rel_err = []
        self.p00 = np.random.randn(hidden_nodes)
        self.p01 = np.random.randn(hidden_nodes)
        self.p1 = np.random.randn(hidden_nodes)
        self.p2 = np.random.randn(hidden_nodes)
        self.m_t = np.array([np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes),
                             np.zeros(hidden_nodes)])


    def sgd_mt(self, w, g_t, theta_0):
        #gradient descent with momentum
        self.m_t[w] = self.beta * self.m_t[w] + (1-self.beta) * g_t
        theta_0 = theta_0 - (self.alpha*self.m_t[w])
            
        return theta_0
        

    def train(self, itr=1000):

        start=len(self.cost_rate)-1
        if start<1:
            start+=1
            self.cost_rate.append(cost(self.points,self.boundary_points,self.p00, self.p01, self.p1, self.p2))
            if self.if_rel_err:
                self.rel_err.append(relative_err(self.p00, self.p01, self.p1, self.p2, 
                                                 all_points=np.vstack([self.points, self.boundary_points])))

        i = start
        while i < start+itr:
            mini_batch_points, mini_batch_boundary = get_mini_batches(self.points, 
                                                                      self.boundary_points, self.batch_size)

            for mini_point, mini_boundary in zip(mini_batch_points, mini_batch_boundary):

                g_w0 = dE_dwj(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2, 0)
                g_w1 = dE_dwj(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2, 1)
                g_u = dE_du(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2)
                g_v = dE_dv(mini_point, mini_boundary, self.p00, self.p01, self.p1, self.p2)

                self.pp00 = self.sgd_mt(0, g_w0, self.p00)
                self.p01 = self.sgd_mt(1, g_w1, self.p01)
                self.p1 = self.sgd_mt(2, g_u, self.p1)
                self.p2 = self.sgd_mt(3, g_v, self.p2)

            self.cost_rate.append(cost(self.points,self.boundary_points,self.p00, self.p01, self.p1, self.p2))
            cost_diff = self.cost_rate[i]-self.cost_rate[i+1]
            if self.if_rel_err:
                self.rel_err.append(relative_err(self.p00, self.p01, self.p1, self.p2, 
                                                 all_points=np.vstack([self.points, self.boundary_points])))
                rel_diff = self.rel_err[i]-self.rel_err[i+1]

            i+=1
                
                
    def save_result(self, output_name=''):
        timestr = time.strftime("%Y%m%d-%H%M")
        np.savez('output/'+ timestr + '_' + output_name +'_nn_params.npz', self.p)
        np.savez('output/'+ timestr + '_' + output_name +'_cost_rate.npz', self.cost_rate)
        if self.if_rel_err:
            np.savez('output/'+ timestr + '_' + output_name +'_rel_err.npz', self.rel_err)

# Graphs

In [24]:
%matplotlib notebook
%matplotlib notebook

## Hidden Nodes

In [25]:
def plot_nodes(a, save_name=None, save=True, zlim=True):
    p = a.p
    hmax = a.p[0][0].shape[0]
    col = int(np.sqrt(hmax)) if int(np.sqrt(hmax)) < 10 else 10
    row = np.ceil(hmax / col)
    
    d_x=0.1

    fig = plt.figure(figsize=(col*3, row*3), dpi=100)
    for h in range(1,hmax+1):
        ax = fig.add_subplot(row,col,h, projection='3d')
        x = y = np.arange(0, 1+d_x, d_x)
        X, Y = np.meshgrid(x, y)

        zs = np.array([p[1][h-1]*sig(z(x,y,p))[h-1] for x,y in zip(np.ravel(X), np.ravel(Y))])
        Z = zs.reshape(X.shape)

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        if zlim:
            ax.set_zlim(0,1)

        ax.plot_surface(X, Y, Z, cmap=cm.coolwarm)
        ax.title.set_text('HN: ' + str(h) + ', v=' + "{:.4E}".format(p[1][h-1]))
        time.sleep(0.2)

    if save==True:
        if save_name is None: 
            plt.savefig('t'+str(T)+'_h'+str(hmax)+'_n'+str(a.nx)+'.png')
        else:
            plt.savefig(save_name)
    else:
        plt.show()

## Error vs Iterations

In [26]:
def plot_iteration(arr, min_it=0, max_it=None):
    plt.figure()
    plt.plot(np.arange(len(arr[min_it:max_it])), np.array(arr[min_it:max_it]))
    plt.show()

## Solution Accuracy on the domain

In [27]:
def accuracy(p):
    d_x=0.01
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = y = np.arange(0, 1.0+d_x, d_x)
    X, Y = np.meshgrid(x, y)

    zs = np.array([psi_a(x,y)-psi_t(x,y,p) for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm_r)
    # ax.ticklabel_format(style='sci', axis='z', scilimits=(0,0), useOffset=True, useMathText=True)


    ax.set_xlabel('x')
    ax.set_ylabel('y')
#     ax.set_zlabel('Accuracy')


    plt.show()

In [28]:
def plot_fun(fun, zlim=False, title=None, return_ax=False, **kwargs):
    d_x=0.01
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x = y = np.arange(0, 1.0+d_x, d_x)
    X, Y = np.meshgrid(x, y)

    zs = np.array([fun(x,y,**kwargs) for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap=cm.coolwarm_r)
    # ax.ticklabel_format(style='sci', axis='z', scilimits=(0,0), useOffset=True, useMathText=True)


    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.title.set_text(title)
    
    if zlim:
        ax.set_zlim(0,1)

    if return_ax:
        return ax
    else:
        plt.show()

# Training

In [29]:
K=10
a = NNTrain(nx=60, bx=300, hidden_nodes=15, alpha=1e-4, batch_size=50, if_rel_err=True)

In [31]:
profile.run("a.train(10)", sort="time")

         22745 function calls (20865 primitive calls) in 12.563 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1840    6.369    0.003    6.369    0.003 <ipython-input-19-534c0b774dd9>:1(dE_dwj)
      920    2.555    0.003    2.555    0.003 <ipython-input-20-6bc3f17db06d>:1(dE_du)
      920    1.663    0.002    1.663    0.002 <ipython-input-21-66e8fa2e80a2>:1(dE_dv)
       10    0.753    0.075    0.753    0.075 <ipython-input-17-db81e09ebaab>:1(cost)
       10    0.658    0.066    0.658    0.066 <ipython-input-18-7b4603c3c0c7>:5(relative_err)
       20    0.250    0.013    0.251    0.013 {method 'shuffle' of 'numpy.random.mtrand.RandomState' objects}
        1    0.163    0.163   12.562   12.562 <ipython-input-23-5601977ab35a>:36(train)
     3680    0.122    0.000    0.122    0.000 <ipython-input-23-5601977ab35a>:28(sgd_mt)
       20    0.011    0.001    0.026    0.001 shape_base.py:739(array_split)
  1930/50    0.003 

In [299]:
profile.run("a.train(10)", sort="time")

         2308275 function calls (2215195 primitive calls) in 9.592 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    1.443    0.144    2.932    0.293 <ipython-input-295-ce93de07c968>:2(relative_err)
   134560    1.405    0.000    1.405    0.000 <ipython-input-217-b58300938a6c>:1(d_dwj_dN2_dxk2)
     1840    1.237    0.001    3.413    0.002 <ipython-input-231-ed990ca71b6d>:2(dE_dwj)
   134560    1.057    0.000    1.057    0.000 <ipython-input-221-174e306cbd11>:1(error_term1)
    79240    0.611    0.000    0.611    0.000 <ipython-input-218-d44fc550655f>:1(d_du_dkN)
      920    0.602    0.001    1.551    0.002 <ipython-input-232-66cd483308c5>:2(dE_du)
      920    0.593    0.001    1.303    0.001 <ipython-input-233-cf49257389f8>:2(dE_dv)
    91200    0.396    0.000    0.835    0.000 linalg.py:2363(norm)
    79240    0.374    0.000    0.374    0.000 <ipython-input-219-025928abb482>:1(d_dv_dkN)
    91200    0.304    

In [335]:
profile.run("a.train(10)", sort="time")

         711890 function calls (710008 primitive calls) in 6.335 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   134560    1.284    0.000    1.284    0.000 <ipython-input-217-b58300938a6c>:1(d_dwj_dN2_dxk2)
     1840    1.162    0.001    3.141    0.002 <ipython-input-231-ed990ca71b6d>:2(dE_dwj)
   134560    0.955    0.000    0.955    0.000 <ipython-input-221-174e306cbd11>:1(error_term1)
      920    0.566    0.001    1.214    0.001 <ipython-input-233-cf49257389f8>:2(dE_dv)
      920    0.557    0.001    1.409    0.002 <ipython-input-232-66cd483308c5>:2(dE_du)
    79240    0.547    0.000    0.547    0.000 <ipython-input-218-d44fc550655f>:1(d_du_dkN)
    79240    0.340    0.000    0.340    0.000 <ipython-input-219-025928abb482>:1(d_dv_dkN)
       11    0.206    0.019    0.206    0.019 <ipython-input-330-0587b893b07d>:5(relative_err)
       11    0.199    0.018    0.199    0.018 <ipython-input-223-db81e09ebaab>:1(cost)
    

In [542]:
profile.run("a.train(10)", sort="time")

         22745 function calls (20865 primitive calls) in 3.013 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1840    1.482    0.001    1.482    0.001 <ipython-input-539-534c0b774dd9>:1(dE_dwj)
      920    0.617    0.001    0.617    0.001 <ipython-input-514-6bc3f17db06d>:1(dE_du)
      920    0.403    0.000    0.403    0.000 <ipython-input-515-66e8fa2e80a2>:1(dE_dv)
       10    0.178    0.018    0.178    0.018 <ipython-input-512-7b4603c3c0c7>:5(relative_err)
       10    0.176    0.018    0.176    0.018 <ipython-input-505-db81e09ebaab>:1(cost)
       20    0.083    0.004    0.083    0.004 {method 'shuffle' of 'numpy.random.mtrand.RandomState' objects}
     3680    0.034    0.000    0.034    0.000 <ipython-input-535-86f0d6c4addd>:35(sgd_mt)
        1    0.030    0.030    3.013    3.013 <ipython-input-535-86f0d6c4addd>:43(train)
       20    0.003    0.000    0.009    0.000 shape_base.py:739(array_split)
  1930/50    

In [ ]:
profile.run("a.train(10)", sort="time")

In [720]:
BC=10
a.alpha = 1e-5
# p=[]
# p.append(np.copy(a.p))
# %time a.train(120)

In [714]:
thresh=1e-5

for i in range(10000):
#     p.append(np.copy(a.p))
    if a.alpha < 1e-9  or BC > 10000:
        break
    if np.abs(np.mean([s-t for s, t in zip(a.cost_rate[-101:], a.cost_rate[-100:])])) < thresh:
        BC=BC*10
        a.alpha=a.alpha/10
    a.train(100)

KeyboardInterrupt: 

Etc

In [716]:
print('alpha:', a.alpha)
print('BC:', BC)
print('relative error:', relative_err_without_points(a.p00, a.p01, a.p1, a.p2, nx=100))

alpha: 0.0001
BC: 10
relative error: 1.4576355291494854


In [ ]:
plot_iteration(a.cost_rate, min_it=100)

In [ ]:
plot_iteration(a.rel_err, min_it=1)

In [ ]:
plot_fun(N, p00=a.p00, p01=a.p01, p1=a.p1, p2=a.p2, title='Neural Network Solution')

In [35]:
relative_err(a.p, nx=200)

0.03732035812236606

In [30]:
a.p = np.load('output/20191223-1714_eq1_2_nn_params.npz')['arr_0']

In [36]:
a.save_result('eq1_2_v1')

timestr = time.strftime("%Y%m%d-%H%M")
np.savez('output/'+ timestr + '_eq1_2_v1_momentum.npz', a.m_t)
np.savez('output/'+ timestr + '_eq1_2_v1_p_list.npz', p)